In [1]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

In [3]:
from collections import Counter

all_addres_cnt = Counter()

for tr in src.tr.iter.iter_tr():
    for op in tr['ops']:
        all_addres_cnt[op['sender']] += 1
        if 'receiver' in op:
            all_addres_cnt[op['receiver']] += 1

len(all_addres_cnt)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.76it/s]

3024

In [9]:
accounts_metadata = {}
# accounts_metadata = src.utils.read_json(src.config.tzktio_accounts_metadata_file)

In [ ]:
from tqdm import tqdm
import requests

for addr, _ in tqdm(all_addres_cnt.most_common()):
    if addr in accounts_metadata:
        continue

    req = requests.get(f'https://api.tzkt.io/v1/accounts/{addr}/metadata')
    if req.status_code == 200:
        accounts_metadata[addr] = req.json()
    elif req.status_code == 204:
        accounts_metadata[addr] = None
    else:
        print(addr, req.status_code, req.text)

 46%|█████████████████████████████████████████████████████████████████▌                                                                            | 1397/3024 [05:55<07:25,  3.65it/s]

In [ ]:
cnt = 0
for addr, meta in accounts_metadata.items():
    if meta != None:
        cnt += 1
print(cnt / len(accounts_metadata))

In [ ]:
src.utils.write_json(accounts_metadata, src.config.tzktio_accounts_metadata_file)